# The Home Depot Decor Case

Getting Started  |  Data Prep  |  Data Exploration  |  Preprocessing  |  **Model Selection & Tuning**  |  Final Model

In [56]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

import gensim

from gensim.models import KeyedVectors as kv

from nltk.corpus import stopwords
from sklearn import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from collections import Counter

# other imports
%matplotlib inline

In [2]:
import sys
import inspect
sys.path.insert(0, '../modules')


# now read in new functions
from helpers import read_in_dataset, get_num_of_levels, flatten_categories, search_cons_status
from hierarchy import Hierarchy
from textpreprocessor import TextPreprocessor

**Read in Data**

In [3]:
# Read in product related data
verbose_opt = False
catalog = read_in_dataset('Decor_catalog.csv', verbose=verbose_opt)
#prod_desc = read_in_dataset('Product_name_description.csv', verbose=verbose_opt)
#prod_engagement = read_in_dataset('Product_engagement.csv', verbose=verbose_opt)

# Read in search related data
#navigations = read_in_dataset('Visual_navigations.csv', verbose=verbose_opt)
search_imp = read_in_dataset('Search_impression.csv', verbose=verbose_opt)

# Overview
----------Data---------------
1. Get Product Hierarchy Dictionary
2. Preprocess Search Terms
3. Consolidate Search Terms

----------Recommender------------ 

_Query Understanding_
4. Search similarity Matrix
  - Get Search Product Matrix
  - Compute Cosine Similarity
  
5. Match Search Term
  - Get Top Recommendation
  
_Query Classification_
6. Get Search-Cat Map
7. Get Top N Nodes
8. Classify Node
9. Make Recommendations


# Get Hierarchy

## Hierarchy Class

In [3]:
#read in data
catalog = read_in_dataset('Decor_catalog.csv', verbose=False)

In [4]:
# Get Product Taxonomy 
h = Hierarchy(data=catalog, num_levels=6)

# Preprocess & Consolidate Search Terms

In [5]:
# Read in Data 
search_imp = read_in_dataset('Search_impression.csv', verbose=False)

In [6]:
# Search
search_corpus = search_imp['Search_term'].values


tp = TextPreprocessor()

# Use preprocessor on searches
search_docs = tp.clean_text(search_imp,'Search_term')

#Consolidate Search after Preprocessing
consolidated_search = tp.compare_clean_searches(search_docs, search_imp)

#searches_trigram = search.trigram_model(clean_searches, verbose=False)
#searches_vec, searches_matrix = tp.vectorize(search_docs, format='docs')

# Query Understanding

In [7]:
catalog = read_in_dataset('Decor_catalog.csv', verbose=False)

In [8]:
# get Search Term Similarity Matrix

# Get impresions as document of sku ids
search_imp['Impression'] = search_imp['Impression'].str.replace(';', ' ')

# impressions array
imp = search_imp['Impression'].values

# Vectorize Data
vec = CountVectorizer()

#Fit Transform
seach_prod = vec.fit_transform(imp)

# Preview Sparse Matrix
pd.DataFrame(seach_prod.toarray(), index=search_imp['Search_term'].values, columns=vec.get_feature_names()).head()

,100012014,100022800,100023109,100023973,100036137,100044505,100048075,100051570,100061089,100074869,...,307717049,307717219,307717221,307727052,307833295,307920434,307939445,307939707,307940057,307940314
coffee,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
bag chair,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
kitchen wall tile,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
mirror tile,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
entryway,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
#Get Search Term Similarity Matrix
search_similarity_matrix = cosine_similarity(seach_prod,seach_prod)

#Dataframe
index = search_imp['Search_term'].values
search_similarity_matrix_df = pd.DataFrame(search_similarity_matrix, index=index, columns=index)

In [10]:
# get term to index
def term2index_dict(search_terms_lst):
    d = {}
    for i,term in enumerate(search_terms_lst):
        d[term] = i
        
    return d

term2index = term2index_dict(search_imp.index.values)

#get top n closest query
def get_topn_query(term, sim_matrix=search_similarity_matrix_df, n=1):
    return sim_matrix.loc[:,term].sort_values(ascending=False).head(n)

In [11]:
get_topn_query('coffee')

coffee    1.0
Name: coffee, dtype: float64

# Query Classification

1. Get Search Term-Cat Matrix: Extract Categories from Top 24 Products 
2. Get Top N Nodes
3. Flatten: Create Node/Category Vectors with Node and Children*
4. Compute Cosine Similarity*
5. Take Top Match (Node/Category)*


In [12]:
#get search terms to limit catelog 
search_labels = pd.Series(vec.get_feature_names(), name='prod')
search_labels.index = search_labels

In [13]:
# Expand Path Columns(e.g. Furniture>Living Room>Accent Tables)

catalog['Category'] = catalog['Category'].str.split('>')

cat = catalog['Category'].values

# Vectorize Product Categories (SKU-ID - Category Matrix)
def dummy(doc):
    return doc

vec_cat = CountVectorizer(lowercase=False, preprocessor=dummy,tokenizer=dummy)

#Fit Transform
prod_cat = vec_cat.fit_transform(cat)


#Get category list 
cat_labels = vec_cat.get_feature_names()

#reduce catalog to product that are in our top 24 product list across search terms
catalog_reduced = catalog[catalog['SKU_ID'].isin(search_labels)]

#reset index for impressions and catelog data
search_imp.set_index('Search_term', inplace=True)
catalog_reduced.set_index('SKU_ID', inplace=True)

In [14]:
#search terms list for for eligible searches
search_terms_unique = list(search_imp.index.unique())

In [15]:
#split impressions into list
search_imp['Impression'] = search_imp['Impression'].str.split()

In [16]:
'''
Get dictionary of {term1: cat1: count, Cat2, count},
                    term2: {cat4: count, Cat9, count}...}
''' 
def get_labels(prod_lst):
    
    labels = []
    for prod in prod_lst:
        lst = catalog_reduced.loc[int(prod)]
        
        if len(lst)>1:
            lsts = catalog_reduced.loc[int(prod)]['Category'].to_list()
            for l in lsts:
                labels.extend(l)
        else:
            labels.extend(lst['Category'])
        
    d = Counter(labels)
    
    return d 

def get_term_cat_dict(search_term_lst):
    term_cat_dict = {}
    for term in search_term_lst:
        term_imp = search_imp.loc[term]['Impression']
        labels = get_labels(term_imp)
        
        term_cat_dict[term] = labels
        
    return term_cat_dict

In [17]:
term_cat_dict = get_term_cat_dict(search_terms_unique)

In [ ]:
def flatten_nodes(candidates=candidates, n=5):
    
    keys = list(candidates.keys())
    
    candiate_cats = keys[:n]
    
    candidate_docs = {}
    
    for candidate in candiate_cats:
        #get node id
        id = h.node2id[candidate]
        
        #get children node ids
        children_ids = h.prod_map[id]
        
        #convert back to children names
        child_doc= ' '.join([h.id2node[child_id] for child_id in children_ids])
        
        candidate_docs[candidate] = child_doc
    
    return candidate_docs

# Recommendation

In [120]:
query = 'accent table'

In [121]:
#Example
candidates = term_cat_dict[query]
candidates

Counter({'Furniture': 27,
         'Living Room Furniture': 24,
         'Accent Tables': 24,
         'End Tables': 21,
         'Entryway Furniture': 3,
         'Entryway Tables': 3,
         'Console Tables': 3})

In [122]:
#save candidate docs
candidate_docs = flatten_nodes()
candidate_docs

{'Furniture': 'Furniture Accessories & Replacement Parts Kids & Baby Furniture Folding Tables & Chairs Home Office Furniture Entryway Furniture Living Room Furniture Kitchen & Dining Room Furniture Bedroom Furniture',
 'Living Room Furniture': 'Accent Tables Chairs Futons Slipcovers Sectionals Ottomans Media Storage TV Stands Living Room Sets Sofas & Loveseats',
 'Accent Tables': 'End Tables Indoor Plant Stands Console Tables Coffee Tables',
 'End Tables': 'nan',
 'Entryway Furniture': 'Umbrella Holders Coat Racks Hall Trees Entryway Benches & Trunks Entryway Tables nan'}

In [79]:
[query] + list(candidate_docs.values())

['accent table',
 'Furniture Accessories & Replacement Parts Kids & Baby Furniture Folding Tables & Chairs Home Office Furniture Entryway Furniture Living Room Furniture Kitchen & Dining Room Furniture Bedroom Furniture',
 'Accent Tables Chairs Futons Slipcovers Sectionals Ottomans Media Storage TV Stands Living Room Sets Sofas & Loveseats',
 'End Tables Indoor Plant Stands Console Tables Coffee Tables',
 'nan',
 'Umbrella Holders Coat Racks Hall Trees Entryway Benches & Trunks Entryway Tables nan']

In [80]:
#Preprocess docs
t = TextPreprocessor()

# Use preprocessor on searches
node_docs = t.clean_docs([query] + list(candidate_docs.values()))

#vectorize
vec_cand, matrix_cand = t.vectorize(node_docs)

In [82]:
node_term = pd.DataFrame(matrix_cand.toarray(), index=[query] +list(candidate_docs.keys()), columns=vec_cand.get_feature_names())
node_term

,accent,accessory,baby,bedroom,bench,chair,coat,coffee,console,dining,...,set,slipcover,sofa,stand,storage,table,tree,trunk,tv,umbrella
accent table,0.848083,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.529863,0.000000,0.000000,0.000000,0.000000
Furniture,0.000000,0.12503,0.12503,0.12503,0.000000,0.102526,0.000000,0.000000,0.000000,0.12503,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.064056,0.000000,0.000000,0.000000,0.000000
Living Room Furniture,0.222157,0.00000,0.00000,0.00000,0.000000,0.222157,0.000000,0.000000,0.000000,0.00000,...,0.270918,0.270918,0.270918,0.222157,0.270918,0.138799,0.000000,0.000000,0.270918,0.000000
Accent Tables,0.000000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.352788,0.352788,0.00000,...,0.000000,0.000000,0.000000,0.289292,0.000000,0.542229,0.000000,0.000000,0.000000,0.000000
End Tables,0.000000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Entryway Furniture,0.000000,0.00000,0.00000,0.00000,0.293299,0.000000,0.293299,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.150265,0.293299,0.293299,0.000000,0.293299


In [84]:
#Get similiarity matrix
query_node_sim_matrix = cosine_similarity(matrix_cand,matrix_cand)

query_node_sim_df = pd.DataFrame(query_node_sim_matrix, index=node_term.index, columns=node_term.index)
query_node_sim_df

,accent table,Furniture,Living Room Furniture,Accent Tables,End Tables,Entryway Furniture
accent table,1.000000,0.033941,0.261952,0.287307,0.00000,0.079620
Furniture,0.033941,1.000000,0.099999,0.034733,0.00000,0.058943
Living Room Furniture,0.261952,0.099999,1.000000,0.139529,0.00000,0.020857
Accent Tables,0.287307,0.034733,0.139529,1.000000,0.00000,0.081478
End Tables,0.000000,0.000000,0.000000,0.000000,1.00000,0.240510
Entryway Furniture,0.079620,0.058943,0.020857,0.081478,0.24051,1.000000


In [100]:
query_node_sim_df.loc[:,query].sort_values(ascending=False)

accent table             1.000000
Accent Tables            0.287307
Living Room Furniture    0.261952
Entryway Furniture       0.079620
Furniture                0.033941
End Tables               0.000000
Name: accent table, dtype: float64

In [101]:
query_node_sim_df.loc[:,query].sort_values(ascending=False).index[1]

def predict_node():
    node_pred = query_node_sim_df.loc[:,query].sort_values(ascending=False).index[1]
    return node_pred
    
def get_recommendations(node_pred):
    
    display_pred = []
    
    #get node id
    node_id = h.node2id[node_pred]
    
    #get children ids
    children_ids = h.prod_map[node_id]
    
    #get children names
    for child_id in children_ids:
        child_name = h.id2node[child_id]
        
        display_pred.append(child_name)
    return display_pred

In [115]:
node_pred = predict_node()

recs = get_recommendations(node_pred)

print(f'Search Term:{query}')
print(f'Predicted Category: {node_pred}')
print(f'Recommended Display Names: {recommendations}')

Search Term:accent table
Predicted Category: Accent Tables
Recommended Display Names: ['End Tables', 'Indoor Plant Stands', 'Console Tables', 'Coffee Tables']


# Run Rec Engine

In [125]:
def get_recommendation(query, verbose=False):
    candidates = term_cat_dict[query]
    
    candidate_docs =  flatten_nodes()
    
    t = TextPreprocessor()

    # Use preprocessor on searches
    node_docs = t.clean_docs([query] + list(candidate_docs.values()))

    #vectorize
    vec_cand, matrix_cand = t.vectorize(node_docs)
    
    #Convert to Dataframe
    node_term = pd.DataFrame(matrix_cand.toarray(), index=[query] +list(candidate_docs.keys()), columns=vec_cand.get_feature_names())
    
    #Get similiarity matrix
    query_node_sim_matrix = cosine_similarity(matrix_cand,matrix_cand)

    query_node_sim_df = pd.DataFrame(query_node_sim_matrix, index=node_term.index, columns=node_term.index)

    #Get prediction
    node_pred = predict_node()

    recs = get_recommendations(node_pred)
    
    if verbose:
        print(f'Search Term:{query}')
        print(f'Predicted Category: {node_pred}')
        print(f'Recommended Display Names: {recommendations}')
        
    return recs

# Evaluator 

In [211]:
class Evaluator:

    def __init__(self, eval_data_file=None, metric=None):
        self.eval_data_file = eval_data_file
        self.metric=metric
        self.run()

    def load_data(self, data_folder='raw', data_type='csv'):
        if data_type == 'csv':
            df = pd.read_csv('../data/{}/{}'.format(data_folder, self.eval_data_file))
        self.eval_data = df

    def get_xy(self):
        self.eval_target = self.eval_data.groupby('Search_term')['Display_name'].apply(list)
        self.eval_terms = self.eval_target.index.tolist()
        
    def jaccard_sim(self, query, recs_pred):
        
        recs_act = self.eval_target.loc[query]
        
        intersect = len(list(set(recs_pred) & set(recs_act)))
        union = len(recs_pred) + len(recs_act) - intersect
        
        score = intersect/union
        
        return score
    
            
    def run(self):
        self.load_data()
        self.get_xy()

In [212]:
e=Evaluator('Visual_navigations.csv')

In [213]:
def mean_hit_rate():
    scores = 0

    for term in e.eval_terms:

        recs_pred =  get_recommendation(term)

        score = e.jaccard_sim(term, recs_pred)

        scores += score
        
    return scores/len(e.eval_terms)

In [214]:
mean_hit_rate()

0.016070307595731324

# Archive

In [194]:
documents = names_trigram

In [195]:
# Word2Vec
model = gensim.models.Word2Vec(documents, size=150, window=10, min_count=5, workers=10)
model.train(documents,total_examples=len(documents),epochs=10)

(15653917, 21696040)

In [201]:
model.most_similar('wall')

/Users/Jaime/miniconda3/envs/thddecor/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('beautiful', 0.32953429222106934),
 ('rainy_day', 0.3221852481365204),
 ('inevitable', 0.32087400555610657),
 ('map_michael', 0.3176816701889038),
 ('outer_size', 0.2966635525226593),
 ('ten', 0.29357579350471497),
 ('freeform', 0.2808625400066376),
 ('painting', 0.2807919383049011),
 ('windmill', 0.2773647904396057),
 ('repeat', 0.27722445130348206)]

In [12]:
# get index and col labels
cat_labels = prod_cat_df.columns.to_list()

In [16]:
# Similarity Matrix of Products
 
def get_prod_similarity(prod1, prod2, categories):
    cat1 = categories[prod1]
    cat2 = categories[prod2]
    
    sum_xy, sum_xx, sum_yy = 0, 0, 0
    
    for i in range(len(cat1)):
        x = cat1[i]
        y = cat2[i]
        
        sum_xx += x * y
        sum_yy += y * y
        sum_xy += x * y
        
    return sum_xy/math.sqrt(sum_xx*sum_yy)

In [24]:
# create empty df
prod_similarity_matrix = pd.DataFrame(index=cat_labels, columns=cat_labels)

In [26]:
class Recommender: 
    def __init__(self, data, prod_col, search_terms, products, cf_method='item', similarity='pearson'):
        self.data = data
        self.search_terms = search_terms
        self.products = products
        self.cf_method = cf_method
        self.similarity = similarity
        self.similarity_matrix = []
        self.term_prod_scores = []
        self.recs = []
        
    def create_similarity_matrix(self):
        '''Create correlation/similarity matrix for all items and stores result as self.similarity_matrix'''
        
        self.similarity_matrix = self._create_empty_df(self.cf_method)
        self._fill_similarity_matrix(self.similarity_matrix, self.similarity)
        
    def _create_empty_df(self, cf_type):
        '''creates and returns empty df with items as rows and columns'''
        if cf_type == 'item':
            labels = self.item_cols
        elif cf_type == 'user':
            labels = self.data[user_col]
        else:
            raise ValueError('Invalid collaborative filtering technique.  Please specify "item" or "user".')
        return pd.DataFrame(index=labels, columns=labels)
    
    def _fill_similarity_matrix(self, similarity_matrix, similarity):
        '''calculates correlation between items using specified similarity and saves results in similarity_matrix
           valid similarity types: jaccard, pearson, cosine'''
        k=0
        item_df = self.data[self.item_cols] 
        #print(item_df)
        progress_bar = tqdm(total = similarity_matrix.shape[0], mininterval=5)
        for i in range(similarity_matrix.shape[0]):
            progress_bar.update()
            similarity_matrix.ix[i,i] = 1.0
            x = item_df.ix[:,i]
            for j in range(i,similarity_matrix.shape[1]):
                y = item_df.ix[:,j]
                similarity_matrix.ix[i,j] = self._get_similarity(x, y, similarity)
                similarity_matrix.ix[j,i] = similarity_matrix.ix[i, j]
                
    def _get_similarity(self, x, y, similarity):
        '''calculated specified correlation between two vectors and returns result'''
        if similarity == 'pearson':
            return self._pearson_similarity(x, y)
        elif similarity == 'jaccard':
            return self._jaccard_similarity(x, y)
        elif similarity == 'cosine':
            return self._cosine_similarity(x, y)
        else:
            raise ValueError('Invalid similarity type.  Please specify "cosine", "pearson", or "jaccard".')
        
    def _pearson_similarity(self, x, y):
        '''returns pearson correlation between x and y: covariance(x,y)/(std_dev(x)*std_dev(y))'''
        #effective if data can be transformed to normal distribution 
        pass

    def _jaccard_similarity(self, x, y):
        '''returns jaccard correlation between x and y: |intsection(x,y)|/|union(x,y)|'''
        #ideal for binary data, e.g. buy vs non-buy
        nonzero_x = set(np.nonzero(x)[0])
        nonzero_y = set(np.nonzero(y)[0])
        intersection_size = len(nonzero_x.intersection(nonzero_y))
        union_size = len(nonzero_x.union(nonzero_y))
        if union_size == 0:
            return 0
        else:
            return intersection_size/union_size

    def _cosine_similarity(self, x, y):
        '''returns cosine of angles between x and y'''
        pass
    

NameError: name 'cosine_similarity' is not defined

In [ ]:
rec_engine.create_similarity_matrix()
